# Define variables J, z, s,t

In [45]:
import numpy as np
import math
import tensorflow as tf

z = np.arange(0.01,0.99,1/100)
N = len(z)
J = np.arange(0,100,2.0)
print(z,J)


print(np.kron(J,z).shape)

[0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1  0.11 0.12 0.13 0.14
 0.15 0.16 0.17 0.18 0.19 0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28
 0.29 0.3  0.31 0.32 0.33 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42
 0.43 0.44 0.45 0.46 0.47 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56
 0.57 0.58 0.59 0.6  0.61 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7
 0.71 0.72 0.73 0.74 0.75 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84
 0.85 0.86 0.87 0.88 0.89 0.9  0.91 0.92 0.93 0.94 0.95 0.96 0.97 0.98] [ 0.  2.  4.  6.  8. 10. 12. 14. 16. 18. 20. 22. 24. 26. 28. 30. 32. 34.
 36. 38. 40. 42. 44. 46. 48. 50. 52. 54. 56. 58. 60. 62. 64. 66. 68. 70.
 72. 74. 76. 78. 80. 82. 84. 86. 88. 90. 92. 94. 96. 98.]
(4900,)


In [2]:
# generate random floating point values for s and t and F
# S and T
from random import seed
from random import random
import array as arr
#random number generator for s
seed(1)
s = arr.array('f')
t = arr.array('f')
    # generate random numbers between 0-1
for _ in range(10):
    #make s
    value_s = random()
    value_s = -1 + (value_s * (0 +1))
    s.append(value_s)
    #make t given restriction with s
    value_t = - value_s - 1 - 0.01    
    t.append(value_t)
    #loop throygh s, and add t sch that it satisifes s+t < -1

#generate random momentum transfers t E (-1,0)
seed(1)
t_individual = arr.array('f')
# generate random numbers between 0-1
for _ in range(100):
    value = random()
    value = -1 + (value * (0 +1))
    t_individual.append(value)
    

# # Define functions to be used in loss function

In [3]:
###Calculate M given array of z and J, for specific points (s,t)
import scipy
import scipy.integrate as integrate
from scipy.integrate import quad
import scipy.special


def Mst_Mts(s,t,z,J):
    return (M_withoutF(s,t,z,J) - M_withoutF(t,s,z,J))/N

def M_withoutF(s,t,z,J):   # use eq 2.6
    sum = 0
    A= []
    B = []
    for j in range(len(J)):
        for Z in z:
            a = 2/math.pi
            b = a*Z
            c = b*(1+Z*t)*(2+Z*t)
            d = c*np.array(P_J(1+2*Z*t, j))
            e = (1-Z*t)*(1+Z*(s+t))*s*(s+t)
            sum += d/e
            B.append(sum)
       # A.append(B)
    B= np.array(B)
    #A2 = np.array(A)
    return B

def P_J(x, J):
    #A =[]
    #for j in range(len(J)):
    B = scipy.special.lpmv(0, J, x)
    C = 16*math.pi*(2*J +1)
    D = C*B
    return D

def Gamma(z):
    return scipy.special.gamma(z)

def hypergeometric(a,b,c,d):
    return scipy.special.hyp2f1(a,b,c,d)

In [4]:
### K^(alpha)
###input = array of t_i, J_j and m_j
#Y sums over j, K sums over i.
def K_alpha(t, J, z):
    sum = 0
    k = []
    for j2 in J[0:]:
        for z2 in z[0:]:
            
            A = (-2*t/math.pi)
            B = A*z2
            C = B*(2 +t*z2)
            D = C*(P_J(1+2*z2*t, j2))
            sum = D
            k.append(sum)
            sum = 0
    
    return k

In [5]:
def K_Beta(J,z):
    sum =0
    A = []
    for j in J:
        for Z in z:
            sum += (-4/math.pi)*Z**(2)*P_J(1,j)
            A.append(sum/N)
        
    return A

# Use functions above to make arrays to be used in loss function# 

In [6]:
#Make Mst - mts array with s and t as scalars
# Making s and t tuples to loop over Mst_Mts function 
#X = tf.stack([z, J], axis=1)

a = np.transpose(s)
b = np.transpose(t)
M=[]
for i in s[0:]:
    for j in t[0:]:
        M.append(Mst_Mts(i,j,z,J))
        #break    
        
M= np.array(M)   
M =tf.convert_to_tensor(M, tf.float64)
print(np.max(M))

17030.613411678987


In [7]:
K_A = []
for t1 in t_individual[0:]:
    value = K_alpha(t1,J,z)
    #should be appending vectors
    K_A.append(value)
K_A= np.array(K_A)
K_A =tf.convert_to_tensor(K_A, tf.float64)
print(np.max(K_A))

159.41835639657864


In [8]:
K_B = K_Beta(J,z)
K_B= np.array(K_B)
K_B = np.transpose(K_B)
K_B =tf.convert_to_tensor(K_B, tf.float64)
print(np.max(K_B))



-0.0006399999999999999


In [9]:
G = 6.67430*(10**(-11))

# def custom_loss(layer):
#     #y_true = F
#     #y_pred does nothing, only there to make keras work 
#     def Custom_Loss_Function(y_true, y_pred):
#         #defie 8*PI*G 
#         G = 6.67430*(10**(-11))
#         c  = 8*math.pi*G
#         alpha = 1
#         beta = 10

#         # first term
#         Mf = tf.tensordot(M, y_true, axes=1)
#         sum1 = 0
#         for i in Mf[0:]:
#             sum1 += i**2


#         #second term
#         Ka = tf.tensordot(K_A, y_true, axes = 1)
#         sum2 = 0
#         for j in Ka[0:]:
#             sum2 += (j - c)**2

#         #third term
#         Kb = tf.tensordot(K_B, y_true, axes = 1)
#         sum3 = Kb

#         value = sum1 + alpha*sum2 +beta*sum3
#         value = int(value)
#         return value
    
#     return Custom_Loss_Function

In [10]:

# def Custom_Loss_Function(y_true, y_pred):

#     #defie 8*PI*G 
#     G = 6.67430*(10**(-11))
#     c  = 8*math.pi*G
#     alpha = 1
#     beta = 10

#     # first term
#     Mf = tf.tensordot(M, y_true, axes=1)
#     Mf = np.array(tf.unstack(Mf, num=None, axis=0))

#     sum1 = 0
#     for i in Mf[0:]:
#         sum1 +=i**2

#     #second term
#     Ka = tf.tensordot(K_A, y_true, axes = 1)
#     Ka = np.array(tf.unstack(Ka, num=None, axis=0))

#     sum2 = 0
#     for j in Ka[0:]:
#         sum2 += (j - c)**2

#     #third term
#     sum3 = 0
#     Kb = tf.tensordot(K_B, y_true, axes = 1)
#     sum3 = Kb

#     value = sum1 + alpha*sum2 +beta*sum3
    
#     return value


In [41]:

def Custom_Loss_Function(y_true, y_pred):
    
    with open('file.txt', 'a') as f:
        f.write(str(y_pred))
    
    print(y_pred)
    #defie 8*PI*G = 1        !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    G = 6.67430*(10**(-11)) 
    c  = 8*math.pi*G # 1
    alpha = 1
    beta = 10
#     y_pred.set_shape([50,1])    
#     y_pred = tf.unstack(y_pred)
    
    # first term
    Mf = tf.tensordot(M, y_pred, axes=1)
    
    Mf = tf.square(Mf)
    sum1 = tf.reduce_sum(Mf)
   
    
    
    #second term
    Ka = tf.tensordot(K_A, y_pred, axes=1)
    sum2 = 0.0
    Ka = Ka - c
    Ka = tf.square(Ka)
    sum2 = tf.reduce_sum(Ka)
    

    #third term
    sum3 = 0.0
    Kb = tf.tensordot(K_B, y_pred, axes=1)
    sum3 = Kb
    
    value = sum1 + alpha*sum2 +beta*sum3
    return value


In [42]:
# # #testing Custom_Loss_Function
# F = np.ones(50,)
# F = tf.convert_to_tensor(F, tf.float64)
# print(Custom_Loss_Function(F, F))

tf.Tensor(
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1.], shape=(50,), dtype=float64) (50,)
tf.Tensor(226909850005.17917, shape=(), dtype=float64)


In [38]:
# A = [1,2]
# B = [3,4]
# print(tf.tensordot(A,B, axes =1) )

# from keras import backend as K
# A = tf.keras.layers.Dot(axes=1)([A, B])
# print(A)

In [ ]:
# A = [1,2]
# B= [2,3]
# print(tf.tensordot(A,B, axes=0))
# print(np.dot(A,B))

In [46]:
-3.5900e+03 < 0

True